In [13]:
import pandas as pd
import folium
from sklearn.cluster import DBSCAN
import numpy as np

# 1. 파일 경로 (현재 디렉토리 기준 - 공백 포함 정확하게 입력)
file_path = "/home/wsl/code/gongmo/src/data/서울시_광진구_안심이_CCTV_연계_현황.csv"

# 2. CSV 파일 읽기
df = pd.read_csv(file_path, encoding='cp949')

# 3. 안심 주소 → 시설유형, 구역코드, 세부번호 분해
df[['시설유형', '구역코드', '세부번호']] = df['안심 주소'].str.extract(r'([가-힣\-]+)-(\d+)-?(.+)?')

# 4. 수치형 변환
df['CCTV 수량'] = pd.to_numeric(df['CCTV 수량'], errors='coerce').fillna(0).astype(int)
df['위도'] = pd.to_numeric(df['위도'], errors='coerce')
df['경도'] = pd.to_numeric(df['경도'], errors='coerce')

# 5. DBSCAN 클러스터링 (100m 기준)
coords = df[['위도', '경도']].dropna().values
kms_per_radian = 6371.0088
epsilon = 0.1 / kms_per_radian  # 약 100m
db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine')
df['cluster'] = db.fit_predict(np.radians(coords))

# 6. 지도 시각화 (Folium)
m = folium.Map(location=[37.54, 127.08], zoom_start=13)
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=row['CCTV 수량'] * 2,
        popup=f"{row['시설유형']} (cluster: {row['cluster']})",
        color='blue',
        fill=True
    ).add_to(m)

# 7. 결과 저장 (노트북 실행 위치에 저장됨)
m.save("cctv_map.html")
df.to_csv("processed_cctv.csv", index=False, encoding="cp949")

print("✅ 분석 완료! 결과 저장됨: ./cctv_map.html, ./processed_cctv.csv")


✅ 분석 완료! 결과 저장됨: ./cctv_map.html, ./processed_cctv.csv


In [8]:
import os
print(repr(file_path))  # 파일 경로 문자열에 이상한 문자 있는지 확인
print(os.path.exists(file_path))  # 존재 여부 재확인

"/home/wsl/code/gongmo/src/data/'서울시 광진구 안심이 CCTV 연계 현황.csv'"
False
